# Penguin Bit-By-Bit

Just a bit of magic to get world started.

In [1]:
from twisted.internet import asyncioreactor
asyncioreactor.install()
__file__ = None

def setup(scene):
    global SCENE
    SCENE = scene
    
import txppb
d = txppb.run(setup)
d.addBoth(print)

The game is called "penguin bit by bit",
so we should probably have a penguin.
In PPB,
these are called "sprites".
This is the standard terminology for video games.

In [2]:
import ppb

class Penguin(ppb.Sprite):
    pass

We defined a penguin.
In PPB, game objects are called "sprites".
This is common in video game development.

In PPB, sprites automatically find the image if it is in the current directory,
and named the same as the sprite.
I made sure I had a cute little png named `penguin.png`.

Now let's put the penguin riiiiiight in the middle.

In [3]:
SCENE.add(Penguin(pos=(0,0)))

But penguins are not meant to sit still! The penguin should move around.
We have the player control the penguin with the arrow keys.
First, let's map the keys to vectors:

In [4]:
from ppb import keycodes

SPEED = 6

DIRECTIONS = {keycodes.Left: ppb.Vector(-SPEED,0), keycodes.Right: ppb.Vector(SPEED,0),
              keycodes.Up: ppb.Vector(0, SPEED), keycodes.Down: ppb.Vector(0, -SPEED)}

In [5]:
from mzutil import set_in_class

Penguin.direction = ppb.Vector(0, 0)

@set_in_class(Penguin)
def on_update(self, update_event, signal):
    self.position += update_event.time_delta * self.direction

Next let's make sure the penguin has a sense of direction, and it follows it:

Oh, um, well.
The penguin is diligently moving...at zero speed,
precisely nowhere.
Let's manually set the direction to see what happens.

In [7]:
Penguin.direction = DIRECTIONS[keycodes.Up]/10


The direction is up, but a little slowly. This gives enough time to set the penguin's direction back to zero manually.
Let's do that now!

In [8]:
Penguin.direction = ppb.Vector(0, 0)


Phew, that was exciting -- but not what we wanted.
We want the penguin to respond to key presses.
Let's set it to set the direction to the key press,
and back to zero when the key is released.

In [9]:
@set_in_class(Penguin)
def on_key_pressed(self, key_event, signal):
    self.direction += DIRECTIONS.get(key_event.key, ppb.Vector(0, 0))    

@set_in_class(Penguin)
def on_key_released(self, key_event, signal):
    if key_event.key in DIRECTIONS:
        self.direction -= DIRECTIONS.get(key_event.key, ppb.Vector(0, 0))

The Penguin is a bit bored, isn't it?
Maybe we should give it an orange ball to play with.

In [10]:
class OrangeBall(ppb.Sprite):
    pass

Again, I made sure to have an image called `orangeball.png`.
Now let's put the ball on the left side of the screen.

In [11]:
SCENE.add(OrangeBall(pos=(-4, 0)))

Try as it might, the penguin cannot kick the ball.
Let's have the ball move away from the penguin when it approaches.

First, let's define what it means to "kick" the ball.
Kicking the ball means deciding where it is going to be in one second,
and then setting its state to "moving".

At first, we will just move it by having the first update move it to the target position.

In [12]:
OrangeBall.is_moving = False

@set_in_class(OrangeBall)
def kick(self, direction):
    self.target_position = self.position + direction
    self.original_position = self.position
    self.time_passed = 0
    self.is_moving = True

@set_in_class(OrangeBall)
def on_update(self, update_event, signal):
    if self.is_moving:
        self.position = self.target_position
        self.is_moving = False

Let's see how it works -- give the orange ball a good kick up and to the right.

In [13]:
ball, = SCENE.get(kind=OrangeBall)
ball.kick(ppb.Vector(1, 1))

But this does not look great -- the ball just teleports.
We want it to move between the original position
and target position, occupying the points in between.

When it's moving, it will interpolate between where it is,
and where it needs to go.

Naively, we would use linear interpolation. But a cool video game tick is to use an "easing" function.
Here, we use the common "smooth step".

In [14]:
from mzutil import smooth_step

@set_in_class(OrangeBall)
def maybe_move(self, update_event, signal):
    if not self.is_moving:
        return False
    self.time_passed += update_event.time_delta
    if self.time_passed >= 1:
        self.position = self.target_position
        self.is_moving = False
        return False
    t = smooth_step(self.time_passed)
    self.position = (1-t) * self.original_position + t * self.target_position
    return True

OrangeBall.on_update = OrangeBall.maybe_move

Now, let's try kicking it again.

In [15]:
ball, = SCENE.get(kind=OrangeBall)
ball.kick(ppb.Vector(1, -1))

But really, the penguin should be kicking the ball.
When the ball sees that it is colliding with the penguin,
it will kick itself in the opposite direction.
If the penguin has gotten right on top of it,
the ball will choose a random direction.

The update function now calls `maybe_move` and will only check collision
if we are not moving right now.

In [16]:
from mzutil import collide
import random

OrangeBall.x_offset = OrangeBall.y_offset = 0.25

@set_in_class(OrangeBall)
def on_update(self, update_event,signal):
    if self.maybe_move(update_event, signal):
        return
    penguin, = update_event.scene.get(kind=Penguin)
    if not collide(penguin, self):
        return
    try:
        direction = (self.position - penguin.position).normalize()
    except ZeroDivisionError:
        direction = ppb.Vector(random.uniform(-1, 1), random.uniform(-1, 1)).normalize()
    self.kick(direction)

But just kicking a ball around is not that much fun. Let's add a target.

In [17]:
class Target(ppb.Sprite):
    pass

Let's put the target at the right of the screen.

In [18]:
SCENE.add(Target(pos=(4, 0)))

Now, we will want a reward for the penguin,
when it kicks the ball into the target.
How about a fish?

In [19]:
class Fish(ppb.Sprite):
    pass

When the target gets the ball, it should remove it, and create a new ball at the other end of the screen.
Then, it will cause a fish to appear.

In [20]:
@set_in_class(Target)
def on_update(self, update_event, signal):
    for ball in update_event.scene.get(kind=OrangeBall):
        if not collide(ball, self):
            continue
        update_event.scene.remove(ball)
        update_event.scene.add(OrangeBall(pos=(-4, random.uniform(-3, 3))))
        update_event.scene.add(Fish(pos=(random.uniform(-4, -3), random.uniform(-3, 3))))

We want to have the penguin eat the fish.
When the fish sees the penguin, it should vanish.

In [21]:
Fish.x_offset = 0.05
Fish.y_offset = 0.2
@set_in_class(Fish)
def on_update(self, update_event,signal):
    penguin, = update_event.scene.get(kind=Penguin)
    if collide(penguin, self):
        update_event.scene.remove(self)

In [22]:
ball, = SCENE.get(kind=OrangeBall)

@set_in_class(OrangeBall)
def set_target(self, position):
    self.target_position = position
    self.original_position = self.position
    self.time_passed = 0
    self.is_moving = True

ball.set_target(ppb.Vector(0,0))


In [23]:
import time
import math

@set_in_class(Fish)
def on_update(self, update_event, signal):
    penguin, = update_event.scene.get(kind=Penguin)

    if collide(penguin, self):
        update_event.scene.remove(self)
        return
    
    if getattr(self, "start_time", None) is None:
        self.start_time = time.time()
        
    if time.time() - self.start_time > 10:
        d = (penguin.position - self.position)
        if math.hypot(d[0], d[1]) > 0:
            d = d / math.hypot(d[0], d[1]) / 30
            self.position += d

In [ ]:
class Score(ppb.Label):
    pass

In [55]:
d = txppb.run(setup)
d.addBoth(print)
SCENE.add(OrangeBall(pos=(-4, 0)))
SCENE.add(Penguin(pos=(0,0)))
SCENE.add(Target(pos=(4, 0)))

NameError: name 'Target' is not defined

In [46]:
from sdl2.ext import pixels2d
p, = SCENE.get(kind=Penguin)
print(dir(p))
a = pixels2d(p.image) & 0xff000000 != 0
len(a[0])

['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__image__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_offset_value', '_rotation', 'basis', 'bottom', 'center', 'facing', 'image', 'layer', 'left', 'position', 'right', 'rotate', 'rotation', 'size', 'top']


/Users/seva/.virtualenvs/bogame/lib/python3.7/site-packages/ipykernel_launcher.py:4: ExperimentalWarning: 'pixels2d' is in an experimental state.
  after removing the cwd from sys.path.


TypeError: source must be a Sprite or SDL_Surface